In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
import pickle
import time
import random
import datetime

pd.set_option('display.max_columns', None)

In [120]:
#set date variables
yesterday = (datetime.date.today()- datetime.timedelta(days=1)).strftime('%Y-%m-%d')
today = datetime.date.today().strftime('%Y-%m-%d')
year = 20202021

In [121]:
today

'2021-06-24'

In [ ]:
def master_function():
    # bring in goalie game logs 
    infile = open("data/goalie_game_log_df.pickle",'rb')
    goalie_game_log = pickle.load(infile)
    infile.close()
    
    #get goalie stats from prior days games
    get_goalie_data(get_goalie_ids())
    

In [50]:
#game boxscore url
url = 'https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(game_id)

In [ ]:
#roster url for future uses 
'https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season=20142015'

In [2]:
#schedule url, date format: 2018-01-09

def get_yesterdays_game_ids():
    yesterday = datetime.date.today()- datetime.timedelta(days=1)
    url = 'https://statsapi.web.nhl.com/api/v1/schedule?date={}'.format(yesterday.strftime('%Y-%m-%d'))
    r = requests.get(url)
    y_sched = r.json()

    game_id = []
    for game in y_sched['dates'][0]['games']:
        game_id.append(game['gamePk'])
    
    return game_id

In [3]:
get_yesterdays_game_ids()

[2020030323]

In [73]:
#get goalie ids from yesterdays games
def get_goalie_ids():
    goalie_id = {}
    for game_id in get_yesterdays_game_ids():
        url = 'https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(game_id)
        r = requests.get(url)
        box_score = r.json()
        for e in ['home', 'away']:
            temp_id = []
            temp_id.extend(box_score['teams'][e]['goalies'])
            for pid in temp_id:
                name = box_score['teams'][e]['players']['ID'+str(pid)]['person']['fullName']
                goalie_id[name] = pid
    return goalie_id
    

        
    
    

In [74]:
get_goalie_ids()

{'Semyon Varlamov': 8473575, 'Andrei Vasilevskiy': 8476883}

In [99]:
def get_goalie_data(goalie_ids, start_year =year, end_year = year):
    counter = 0
    for name, gid in goalie_ids.items():

        sequence = [x/10 for x in range(60, 120)]
        time.sleep(random.choice(sequence))
        url = 'https://www.naturalstattrick.com/playerreport.php?fromseason={}&thruseason={}&playerid={}&sit=all&stype=2&stdoi=oi&rate=n&v=g'.format(start_year, end_year, gid)
        #due to number of http requests, NST may ban your IP before the loop finishes. I needed to use a VPN to get around this. If IP gets banned, this function will still return the current DF and you can call the function again and pass in an updated goalie dictionary to get the rest
        try:
            individual_df = pd.read_html(url)[0]
            individual_df['Name'] = name
            individual_df['ID'] = gid

        except:
            print(f'Ended before {name}')
            return all_goalies

        if counter == 0:
            all_goalies = individual_df

        elif counter != 0:
            all_goalies = pd.concat([all_goalies, individual_df])



        counter +=1
    
    return all_goalies

In [76]:
def goalie_features(df, rolling_games = 40):
    rolling_games = rolling_games
    min_games = 10
    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    
    df['Rolling_TOI'] = df.groupby('ID')['TOI'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_FA'] = df.groupby('ID')['FA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_SA'] = df.groupby('ID')['SA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_GA'] = df.groupby('ID')['GA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_xGA'] = df.groupby('ID')['xGA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_HDCA'] = df.groupby('ID')['HDCA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_HDGA'] = df.groupby('ID')['HDGA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    
    df['Goalie_FenwickSV%'] =  (df['Rolling_FA'] - df['Rolling_GA']) /  df['Rolling_FA']
    df['Goalie_GSAx'] = df['Rolling_xGA'] - df['Rolling_GA']
    df['Goalie_GSAx/60'] =  df['Goalie_GSAx']*60 /  df['Rolling_TOI']
    df['Goalie_HDCSV%'] = (df['Rolling_HDCA'] - df['Rolling_HDGA'] ) / df['Rolling_HDCA'] 
    return df

In [79]:
infile = open("data/goalie_game_log_df.pickle",'rb')
goalie_game_log = pickle.load(infile)
infile.close()

In [102]:
def update_goalie_logs():
    # bring in goalie game logs 
    infile = open("data/goalie_game_log_df.pickle",'rb')
    goalie_game_log = pickle.load(infile)
    infile.close()
    
    #get goalie stats for goalies that played yesterday
    yesterdays_goalies_df = get_goalie_data(get_goalie_ids())
    yesterdays_goalies_df['Date'] = yesterdays_goalies_df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    
    #concact yesterdays goalies stats for just yesterday's games 
    goalie_game_log = pd.concat([goalie_game_log, yesterdays_goalies_df[yesterdays_goalies_df['Date'] == yesterday]])
    #recalculate stats for features
    goalie_game_log = goalie_features(goalie_game_log)
    
    return goalie_game_log

In [103]:
def get_and_format_nst_team_stats(season, sit, rate):
    #dict to convert team names from NST to team abbreviations from NHL API
    nst_to_sched = {'Anaheim Ducks': 'ANA',
                     'Arizona Coyotes': 'ARI',
                     'Boston Bruins': 'BOS',
                     'Buffalo Sabres': 'BUF',
                     'Calgary Flames': 'CGY',
                     'Carolina Hurricanes': 'CAR',
                     'Chicago Blackhawks': 'CHI',
                     'Colorado Avalanche': 'COL',
                     'Columbus Blue Jackets': 'CBJ',
                     'Dallas Stars': 'DAL',
                     'Detroit Red Wings': 'DET',
                     'Edmonton Oilers': 'EDM',
                     'Florida Panthers': 'FLA',
                     'Los Angeles Kings': 'L.A',
                     'Minnesota Wild': 'MIN',
                     'Montreal Canadiens': 'MTL',
                     'Nashville Predators': 'NSH',
                     'New Jersey Devils': 'N.J',
                     'New York Islanders': 'NYI',
                     'New York Rangers': 'NYR',
                     'Ottawa Senators': 'OTT',
                     'Philadelphia Flyers': 'PHI',
                     'Pittsburgh Penguins': 'PIT',
                     'San Jose Sharks': 'S.J',
                     'St Louis Blues': 'STL',
                     'Tampa Bay Lightning': 'T.B',
                     'Toronto Maple Leafs': 'TOR',
                     'Vancouver Canucks': 'VAN',
                     'Vegas Golden Knights': 'VGK',
                     'Washington Capitals': 'WSH',
                     'Winnipeg Jets': 'WPG'}
    #dyanmic URL
    url = 'https://www.naturalstattrick.com/games.php?fromseason={}&thruseason={}&stype=2&sit={}&loc=B&team=All&rate={}'.format(
        season,
        season,
        sit,
        rate)
    #scrape html table from webpage
    df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
    #reset index
    df.reset_index(inplace = True)
    #format date
    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    #add team game number
    df['Game_Number'] = df.groupby('Team').cumcount() + 1
    #replcate Team name with team abbreviation
    df = df.replace({'Team': nst_to_sched})
    #add team key to merge with game results df
    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    return df

In [104]:
#merge 5v5, PP, and PK team game logs from NST
def merge_team_stats(primary_df, pp_df, pk_df):
    primary_df = primary_df.merge(pk_df[['Team_Key', 'TOI', 'xGA', 'GA']], on = 'Team_Key', how = 'left', suffixes = ('','_pk') )
    primary_df = primary_df.merge(pp_df[['Team_Key', 'TOI', 'xGF', 'GF']], on = 'Team_Key', how = 'left', suffixes = ('','_pp') )
    return primary_df

In [105]:
#calculate team features. Number of rolling games used is dynamic to test if different numbers of game work better.
def calculate_team_features(df, rolling_games = 40):

    df[f'sum_rolling{rolling_games}_TOI_5v5'] = df.groupby('Team')['TOI'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df[f'sum_rolling{rolling_games}_FF_5v5'] = df.groupby('Team')['FF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_FA_5v5'] = df.groupby('Team')['FA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GF_5v5'] = df.groupby('Team')['GF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GA_5v5'] = df.groupby('Team')['GA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGF_5v5'] = df.groupby('Team')['xGF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGA_5v5'] = df.groupby('Team')['xGA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_SF_5v5'] = df.groupby('Team')['SF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'last_{rolling_games}_FF%_5v5'] = df[f'sum_rolling{rolling_games}_FF_5v5']*100/ (df[f'sum_rolling{rolling_games}_FF_5v5']+df[f'sum_rolling{rolling_games}_FA_5v5'])
    df[f'last_{rolling_games}_GF%_5v5'] = df[f'sum_rolling{rolling_games}_GF_5v5']*100/ (df[f'sum_rolling{rolling_games}_GF_5v5']+df[f'sum_rolling{rolling_games}_GA_5v5'])
    df[f'last_{rolling_games}_xGF%_5v5'] = df[f'sum_rolling{rolling_games}_xGF_5v5']*100/ (df[f'sum_rolling{rolling_games}_xGF_5v5']+df[f'sum_rolling{rolling_games}_xGA_5v5'])
    df[f'last_{rolling_games}_SH%'] = df[f'sum_rolling{rolling_games}_GF_5v5']*100 / df[f'sum_rolling{rolling_games}_SF_5v5']
    
    
    #fix NaNs in pp and pk features. If team wasnt on PP or PK in a game that game is missing from dataframe.
    df['TOI_pp'] = np.where(df['TOI_pp'].isna(), 0, df['TOI_pp'])
    df['TOI_pk'] = np.where(df['TOI_pk'].isna(), 0, df['TOI_pk'])
    df['xGF_pp'] = np.where(df['xGF_pp'].isna(), 0, df['xGF_pp'])
    df['GF_pp']  = np.where(df['GF_pp'].isna(), 0, df['GF_pp'])
    df['xGA_pk'] = np.where(df['xGA_pk'].isna(), 0, df['xGA_pk'])
    df['GA_pk']  = np.where(df['GA_pk'].isna(), 0, df['GA_pk'])
    
    #pp features
    df[f'sum_rolling{rolling_games}_TOI_pp'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGF_pp'] = df.groupby('Team')['xGF_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GF_pp'] = df.groupby('Team')['GF_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'last{rolling_games}_pp_TOI_per_game'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
    df[f'last{rolling_games}_xGF_per_min_pp'] = df[f'sum_rolling{rolling_games}_xGF_pp'] / df[f'sum_rolling{rolling_games}_TOI_pp'] 
    df[f'last{rolling_games}_GF_per_min_pp'] = df[f'sum_rolling{rolling_games}_GF_pp'] / df[f'sum_rolling{rolling_games}_TOI_pp'] 
    
    
    #pk features
    df[f'sum_rolling{rolling_games}_TOI_pk'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGA_pk'] = df.groupby('Team')['xGA_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GA_pk'] = df.groupby('Team')['GA_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'last{rolling_games}_pk_TOI_per_game'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
    df[f'last{rolling_games}_xGA_per_min_pk'] = df[f'sum_rolling{rolling_games}_xGA_pk'] / df[f'sum_rolling{rolling_games}_TOI_pk'] 
    df[f'last{rolling_games}_GA_per_min_pk'] = df[f'sum_rolling{rolling_games}_GA_pk'] / df[f'sum_rolling{rolling_games}_TOI_pk'] 
    
    #to get back to back category
    df['Last_Game_Date'] = df.groupby('Team')['Date'].shift()
    df['Days_Since_Last_Game'] = df['Date'] - df['Last_Game_Date']
    df['B2B'] = np.where(df['Days_Since_Last_Game'] == '1 days', 1, 0)
    
    

    
    return df

In [111]:
def get_curent_season_team_stats():
    #scraping team stats from NST
    sequence = [x/10 for x in range(60, 120)]
    time.sleep(random.choice(sequence))

    primarysva = get_and_format_nst_team_stats(year,'sva', 'n')

    time.sleep(random.choice(sequence))

    pp = get_and_format_nst_team_stats(year,'pp', 'n')

    time.sleep(random.choice(sequence))

    pk = get_and_format_nst_team_stats(year,'pk', 'n')
    
    current_df = merge_team_stats(primarysva, pp, pk)
    
    return current_df
    

In [127]:
def get_todays_sched(today = today):
    sched_df = hockey_scraper.scrape_schedule(today,today)
    sched_df['Home_Team_Won'] = np.where(sched_df['home_score'] > sched_df['away_score'], 1, 0)
    #create keys for home and away team in order to import their features
    sched_df['Home_Team_Key'] = sched_df['home_team'].astype(str)+'_'+sched_df['date'].astype(str)
    sched_df['Away_Team_Key'] = sched_df['away_team'].astype(str)+'_'+sched_df['date'].astype(str)
    return sched_df

In [129]:
get_todays_sched()

Scraping the schedule between 2021-06-24 and 2021-06-24


,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key
0,2020030316,2021-06-24,Centre Bell,MTL,VGK,2021-06-25,0,0,Preview,0,MTL_2021-06-24,VGK_2021-06-24


In [113]:
team_stats = pd.read_csv('data/team_stats_all_seasons')

In [115]:
team_stats.tail()

,Unnamed: 0,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Attendance,Date,Game_Number,Team_Key,TOI_pk,xGA_pk,GA_pk,TOI_pp,xGF_pp,GF_pp,sum_rolling40_TOI_5v5,sum_rolling40_FF_5v5,sum_rolling40_FA_5v5,sum_rolling40_GF_5v5,sum_rolling40_GA_5v5,sum_rolling40_xGF_5v5,sum_rolling40_xGA_5v5,sum_rolling40_SF_5v5,last_40_FF%_5v5,last_40_GF%_5v5,last_40_xGF%_5v5,last_40_SH%,sum_rolling40_TOI_pp,sum_rolling40_xGF_pp,sum_rolling40_GF_pp,last40_pp_TOI_per_game,last40_xGF_per_min_pp,last40_GF_per_min_pp,sum_rolling40_TOI_pk,sum_rolling40_xGA_pk,sum_rolling40_GA_pk,last40_pk_TOI_per_game,last40_xGA_per_min_pk,last40_GA_per_min_pk,Last_Game_Date,Days_Since_Last_Game,B2B
11439,1132,"2021-05-16 - Flames 6, Canucks 5",CGY,Limited ReportFull Report,40.966667,54.94,27.60,66.56,36.90,21.92,62.73,24.60,20.30,54.79,5.35,1.84,74.39,1.90,1.16,62.13,20.96,14.73,58.72,7.73,2.57,75.02,5.47,2.57,68.02,3.21,0.92,77.77,58.75,64.28,13.40,11.98,52.81,4.25,10.01,29.80,1.04,0.0,100.0,24.48,100.0,28.38,9.76,74.41,14.90,6.14,70.82,1.12,0.88,55.89,7.5,85.64,21.74,90.93,1.127,0,2021-05-16,38,CGY_2021-05-16,6.816667,0.83,2.0,12.000000,1.67,0.0,1942.066667,1340.57,1221.04,75.53,80.73,68.47,62.11,943.12,52.333103,48.336106,52.435289,8.008525,193.650000,21.44,20.0,4.841250,0.110715,0.103279,188.716667,19.78,22.0,4.717917,0.104813,0.116577,2021-05-13,3 days 00:00:00.000000000,0
11440,1726,"2021-05-18 - Flames 2, Canucks 4",CGY,Limited ReportFull Report,47.416667,39.48,39.43,50.03,32.75,31.46,51.01,28.05,23.10,54.84,1.00,2.92,25.61,2.26,1.35,62.69,24.13,17.34,58.19,9.87,5.85,62.79,8.67,4.98,63.54,1.02,2.90,26.09,11.79,41.80,13.99,11.76,54.33,10.80,9.22,53.95,0.00,0.0,NaN,0.00,100.0,13.73,18.32,42.84,8.85,8.80,50.15,0.00,0.00,NaN,0.0,100.00,3.58,87.37,0.909,0,2021-05-18,56,CGY_2021-05-18,4.200000,0.00,0.0,4.200000,0.32,0.0,1932.350000,1354.00,1193.29,79.87,80.68,68.40,60.28,951.40,53.154529,49.747742,53.155113,8.394997,199.650000,22.87,20.0,4.991250,0.114550,0.100175,193.533333,20.20,24.0,4.838333,0.104375,0.124010,2021-05-16,2 days 00:00:00.000000000,0
11441,1727,"2021-05-18 - Flames 2, Canucks 4",VAN,Limited ReportFull Report,47.416667,39.43,39.48,49.97,31.46,32.75,48.99,23.10,28.05,45.16,2.92,1.00,74.39,1.35,2.26,37.31,17.34,24.13,41.81,5.85,9.87,37.21,4.98,8.67,36.46,2.90,1.02,73.91,58.20,88.21,11.76,13.99,45.67,9.22,10.80,46.05,0.00,0.0,NaN,0.00,100.0,18.32,13.73,57.16,8.80,8.85,49.85,0.00,0.00,NaN,0.0,100.00,12.63,96.42,1.091,0,2021-05-18,56,VAN_2021-05-18,4.200000,0.32,0.0,4.200000,0.00,0.0,1960.916667,1254.41,1454.82,64.37,87.23,67.87,84.83,920.78,46.301348,42.460422,44.446627,6.990812,175.083333,18.29,18.0,4.377083,0.104465,0.102808,207.633333,23.14,25.0,5.190833,0.111446,0.120405,2021-05-16,2 days 00:00:00.000000000,0
11442,1267,"2021-05-19 - Canucks 2, Flames 6",VAN,Limited ReportFull Report,44.483333,31.69,47.34,40.10,26.71,40.01,40.03,16.60,26.90,38.16,1.00,5.89,14.56,1.57,3.31,32.15,17.32,31.32,35.60,5.87,19.33,23.30,3.84,13.38,22.32,1.02,4.84,17.45,26.59,63.85,11.17,11.98,48.25,4.79,6.27,43.30,0.00,1.0,0.0,0.00,84.0,13.28,14.59,47.66,6.90,6.86,50.14,0.00,0.00,NaN,0.0,100.00,6.05,78.08,0.841,0,2021-05-19,42,VAN_2021-05-19,3.483333,0.34,0.0,8.300000,0.16,1.0,1963.383333,1260.16,1447.36,67.29,84.26,67.88,84.29,924.55,46.542962,44.401188,44.608004,7.278135,171.700000,17.74,17.0,4.292500,0.103320,0.099010,204.450000,22.73,24.0,5.111250,0.111176,0.117388,2021-05-18,1 days 00:00:00.000000000,1
11443,1266,"2021-05-19 - Canucks 2, Flames 6",CGY,Limited ReportFull Report,44.483333,47.34,31.69,59.90,40.01,26.71,59.97,26.90,16.60,61.84,5.89,1.00,85.44,3.31,1.57,67.85,31.32,17.32,64.40,19.33,5.87,76.70,13.38,3.84,77.68,4.84,1.02,82.55,36.15,73.41,11.98,11.17,51.75,6.27,4.79,56.70,1.00,0.0,100.0,16.00,100.0,14.59,13.28,52.34,6.86,6.90,49.8

In [ ]:
def add_features


In [ ]:
def concat_current_season_feature_to_master

In [112]:
get_curent_season_team_stats()

,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Attendance,Date,Game_Number,Team_Key,TOI_pk,xGA_pk,GA_pk,TOI_pp,xGF_pp,GF_pp
0,"2021-01-13 - Penguins 3, Flyers 6",PHI,Limited ReportFull Report,51.933333,37.46,47.95,43.86,29.37,37.07,44.21,21.57,29.99,41.84,3.86,2.07,65.12,1.88,1.60,54.15,17.59,19.09,47.95,9.56,7.15,57.21,7.71,5.95,56.44,3.84,1.06,78.38,49.75,82.22,8.03,11.80,40.50,1.94,6.91,21.95,0.00,1.06,0.00,0.00,84.68,15.50,23.31,39.94,9.84,15.21,39.28,0.00,0.00,NaN,0.0,100.00,17.90,93.10,1.110,0,2021-01-13,1,PHI_2021-01-13,5.783333,0.28,1.0,2.133333,0.28,2.0
1,"2021-01-13 - Penguins 3, Flyers 6",PIT,Limited ReportFull Report,51.933333,47.95,37.46,56.14,37.07,29.37,55.79,29.99,21.57,58.16,2.07,3.86,34.88,1.60,1.88,45.85,19.09,17.59,52.05,7.15,9.56,42.79,5.95,7.71,43.56,1.06,3.84,21.62,17.78,50.25,11.80,8.03,59.50,6.91,1.94,78.05,1.06,0.00,100.00,15.32,100.00,23.31,15.50,60.06,15.21,9.84,60.72,0.00,0.00,NaN,0.0,100.00,6.90,82.10,0.890,0,2021-01-13,1,PIT_2021-01-13,2.133333,0.28,2.0,5.783333,0.28,1.0
2,"2021-01-13 - Blackhawks 1, Lightning 5",CHI,Limited ReportFull Report,45.583333,30.32,37.17,44.92,21.50,29.03,42.55,17.82,23.51,43.11,0.00,2.97,0.00,1.03,1.88,35.29,18.48,25.28,42.22,1.93,7.20,21.16,1.90,7.30,20.67,0.00,1.97,0.00,0.00,73.06,16.09,18.55,46.44,11.69,9.45,55.31,0.00,0.98,0.00,0.00,89.66,6.95,9.51,42.23,4.29,6.36,40.31,0.00,0.00,NaN,0.0,100.00,0.00,87.37,0.874,0,2021-01-13,1,CHI_2021-01-13,7.350000,0.93,2.0,6.300000,0.49,1.0
3,"2021-01-13 - Blackhawks 1, Lightning 5",T.B,Limited ReportFull Report,45.583333,37.17,30.32,55.08,29.03,21.50,57.45,23.51,17.82,56.89,2.97,0.00,100.00,1.88,1.03,64.71,25.28,18.48,57.78,7.20,1.93,78.84,7.30,1.90,79.33,1.97,0.00,100.00,26.94,100.00,18.55,16.09,53.56,9.45,11.69,44.69,0.98,0.00,100.00,10.34,100.00,9.51,6.95,57.77,6.36,4.29,59.69,0.00,0.00,NaN,0.0,100.00,12.63,100.00,1.126,0,2021-01-13,1,T.B_2021-01-13,6.300000,0.49,1.0,7.350000,0.93,2.0
4,"2021-01-13 - Canadiens 4, Maple Leafs 5",MTL,Limited ReportFull Report,48.600000,50.47,47.93,51.29,35.13,30.63,53.42,23.29,18.73,55.42,2.11,1.85,53.35,1.92,1.62,54.35,27.02,27.68,49.39,9.91,5.32,65.07,5.47,3.51,60.90,1.06,0.92,53.56,19.36,73.84,17.21,22.39,43.46,7.36,8.61,46.06,1.05,0.00,100.00,14.30,100.00,19.23,18.34,51.18,8.42,6.64,55.92,0.00,0.89,0.0,0.0,86.61,9.07,90.14,0.992,0,2021-01-13,1,MTL_2021-01-13,5.983333,1.64,2.0,5.416667,0.88,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,"2021-05-08 - Blues 1, Golden Knights 4",VGK,Limited ReportFull Report,44.350000,49.24,31.11,61.28,36.23,21.07,63.23,22.61,11.20,66.87,0.94,0.00,100.00,2.22,1.19,65.12,21.07,18.80,52.84,10.83,3.97,73.18,7.95,1.95,80.31,0.94,0.00,100.00,11.84,100.00,10.31,14.47,41.62,3.99,3.90,50.61,0.00,0.00,NaN,0.00,100.00,20.09,11.21,64.18,7.49,5.26,58.73,0.00,0.00,NaN,0.0,100.00,4.17,100.00,1.042,7567,2021-05-08,56,VGK_2021-05-08,4.416667,0.63,0.0,4.600000,0.64,1.0
1732,"2021-05-08 - Avalanche 3, Kings 2",COL,Limited ReportFull Report,48.750000,41.84,31.85,56.78,30.58,21.48,58.73,21.59,13.52,61.49,2.13,1.90,52.85,1.83,1.06,63.22,18.62,14.25,56.65,7.77,4.39,63.92,6.63,4.40,60.15,0.00,0.93,0.00,0.00,78.86,10.97,9.66,53.15,5.27,4.62,53.32,2.09,1.03,67.05,39.67,77.73,15.30,13.42,53.28,7.50,3.63,67.36,0.00,0.00,NaN,0.0,100.00,9.88,85.93,0.958,2482,2021-05-08,56,COL_2021-05-08,4.000000,0.50,0.0,5.183333,0.81,1.0
1733,"2021-05-08 - Avalanche 3, Kings 2",L.A,Limited ReportFull Report,48.750000,31.85,41.84,43.22,21.48,30.58,41.27,13.52,21.59,38.51,1.90,2.13,47.15,1.06,1.83,36.78,14.25,18.62,43.35,4.39,7.77,36.08,4.4

In [107]:
features = pd.read_csv('data/all_games_multirolling_SVA_3.csv')

In [108]:
features.tail()

,Unnamed: 0,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Goalie_FenwickSV%,home_Goalie_GSAx/60,home_Goalie_HDCSV%,away_goalie,away_Goalie_FenwickSV%,away_Goalie_GSAx/60,away_Goalie_HDCSV%,home_Game_Number,home_Team_Key,home_last_1_FF%_5v5,home_last_1_GF%_5v5,home_last_1_xGF%_5v5,home_last_1_SH%,home_last1_pp_TOI_per_game,home_last1_xGF_per_min_pp,home_last1_GF_per_min_pp,home_last1_pk_TOI_per_game,home_last1_xGA_per_min_pk,home_last1_GA_per_min_pk,home_last_3_FF%_5v5,home_last_3_GF%_5v5,home_last_3_xGF%_5v5,home_last_3_SH%,home_last3_pp_TOI_per_game,home_last3_xGF_per_min_pp,home_last3_GF_per_min_pp,home_last3_pk_TOI_per_game,home_last3_xGA_per_min_pk,home_last3_GA_per_min_pk,home_Last_Game_Date,home_Days_Since_Last_Game,home_B2B,home_last_5_FF%_5v5,home_last_5_GF%_5v5,home_last_5_xGF%_5v5,home_last_5_SH%,home_last5_pp_TOI_per_game,home_last5_xGF_per_min_pp,home_last5_GF_per_min_pp,home_last5_pk_TOI_per_game,home_last5_xGA_per_min_pk,home_last5_GA_per_min_pk,home_last_10_FF%_5v5,home_last_10_GF%_5v5,home_last_10_xGF%_5v5,home_last_10_SH%,home_last10_pp_TOI_per_game,home_last10_xGF_per_min_pp,home_last10_GF_per_min_pp,home_last10_pk_TOI_per_game,home_last10_xGA_per_min_pk,home_last10_GA_per_min_pk,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_GF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_last20_GA_per_min_pk,home_last_30_FF%_5v5,home_last_30_GF%_5v5,home_last_30_xGF%_5v5,home_last_30_SH%,home_last30_pp_TOI_per_game,home_last30_xGF_per_min_pp,home_last30_GF_per_min_pp,home_last30_pk_TOI_per_game,home_last30_xGA_per_min_pk,home_last30_GA_per_min_pk,home_last_40_FF%_5v5,home_last_40_GF%_5v5,home_last_40_xGF%_5v5,home_last_40_SH%,home_last40_pp_TOI_per_game,home_last40_xGF_per_min_pp,home_last40_GF_per_min_pp,home_last40_pk_TOI_per_game,home_last40_xGA_per_min_pk,home_last40_GA_per_min_pk,home_last40_pp_TOI_per_game.1,home_last40_xGF_per_min_pp.1,home_last40_GF_per_min_pp.1,home_last40_pk_TOI_per_game.1,home_last40_xGA_per_min_pk.1,home_last40_GA_per_min_pk.1,home_last_50_FF%_5v5,home_last_50_GF%_5v5,home_last_50_xGF%_5v5,home_last_50_SH%,home_last50_pp_TOI_per_game,home_last50_xGF_per_min_pp,home_last50_GF_per_min_pp,home_last50_pk_TOI_per_game,home_last50_xGA_per_min_pk,home_last50_GA_per_min_pk,away_Game_Number,away_Team_Key,away_last_1_FF%_5v5,away_last_1_GF%_5v5,away_last_1_xGF%_5v5,away_last_1_SH%,away_last1_pp_TOI_per_game,away_last1_xGF_per_min_pp,away_last1_GF_per_min_pp,away_last1_pk_TOI_per_game,away_last1_xGA_per_min_pk,away_last1_GA_per_min_pk,away_last_3_FF%_5v5,away_last_3_GF%_5v5,away_last_3_xGF%_5v5,away_last_3_SH%,away_last3_pp_TOI_per_game,away_last3_xGF_per_min_pp,away_last3_GF_per_min_pp,away_last3_pk_TOI_per_game,away_last3_xGA_per_min_pk,away_last3_GA_per_min_pk,away_Last_Game_Date,away_Days_Since_Last_Game,away_B2B,away_last_5_FF%_5v5,away_last_5_GF%_5v5,away_last_5_xGF%_5v5,away_last_5_SH%,away_last5_pp_TOI_per_game,away_last5_xGF_per_min_pp,away_last5_GF_per_min_pp,away_last5_pk_TOI_per_game,away_last5_xGA_per_min_pk,away_last5_GA_per_min_pk,away_last_10_FF%_5v5,away_last_10_GF%_5v5,away_last_10_xGF%_5v5,away_last_10_SH%,away_last10_pp_TOI_per_game,away_last10_xGF_per_min_pp,away_last10_GF_per_min_pp,away_last10_pk_TOI_per_game,away_last10_xGA_per_min_pk,away_last10_GA_per_min_pk,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_GF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_last20_GA_per_min_pk,away_last_30_FF%_5v5,away_last_30_GF%_5v5,away_last_30_xGF%_5v5,away_last_30_SH%,away_last30_pp_TOI_per_game,away_last30_xGF_per_min_pp,away_last30_GF_per_min_pp,away_last30_pk_TOI_per_game,away_last30_xGA_per_min_pk,away_last30_GA_per_min_pk,away_last_40_FF%_5v5,away_last_